# A2C (Advantage Actor-Critic)
- 이는 A3C라는 알고리즘에서 파생된 분산학습형 심층강화학습 알고리즘이다.
- 이는 에이전트를 여러개 사용해서 학습을 진행하며 모든 에이전트가 같은 신경망을 공유한다.
- Advantage 학습은 Q함수를 학습할 때 2단계 이상 미래의 행동가치까지 계산에 넣는 것이 핵심이다.
- Advantage 학습을 위한 수정식 `Q(s_t, a_t) -> R(t+1) + r * R(t+2) + r^2 * max_a[Q(s_t+2, a)]`
- 여기서 주의해야 할 것이 단순히 미래 여러단계를 넣으면 될 수 있겠다 생각할 수 있겠지만 그러면 그 단계 수만큼 행동을 결정해야한다.
- 그 행동을 결정하기 위해 Q함수를 사용하므로 지금은 적절한 행동을 선택하지 못할 확률이 높고 그에 따라 학습이 잘못될 가능성이 있다. 따라서 적절한 단계수를 선택하는 것이 일반적이다.




- Actor-Critic은 정책반복과 가치반복 알고리즘을 조합한 것이다. (Actor의 경우 출력 수 는 행동의 가짓수, Critic의 출력은 상태가치 값)
- 정책 반복이란 정책에 따라 목표에 빠르게 도달했던 경우에 수행했던 행동(action)을 중요한 것으로 보고, 이때의 행동을 앞으로도 취할 수 있도록 정책을 수정하는 방법이다.
    - 소프트맥스 함수, 정책 경사 알고리즘이 있다.
- 가치 반복이란 목표 지점부터 거슬러 올라가며 목표 지점과 가까운 상태로 에이전트를 유도해 오는 방법이다. 즉, 목표지점외의 지점에도 가치를 부여하는 것이다.
    - 가치 반복은 행동가치와 상태가치로 나누어 볼 수 있다.
    - 상태가치란 상태 s에서 정책ㅠ(파이)을 따라 이동할 때 얻으리라 기대할 수 있는 할인 총보상 G_t을 말한다. 일반적으로 이 값은 벨만 방정식을 따른다.
    - 행동가치란 상태 s에서 어떤 행동 a를 했을 때 얻으리라 기대할 수 있는 할인 총보상 G_t를 말한다.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

In [2]:
ENV = 'CartPole-v0'
GAMMA = 0.99
MAX_STEPS = 200
NUM_EPISODES = 1000

NUM_PROCESSES = 32 # 동시 실행 환경 수 (32 agent 사용)
NUM_ADVANCED_STEP = 5 # 총 보상을 계산할 때 Advantage학습을 할 단계 수(미래 5단계까지 고려)
# A2C 손실 함수 계산에 사용되는 상수
value_loss_coef = 0.5
entropy_coef = 0.01
max_grad_norm = 0.5

In [3]:
# 메모리 클래스 정의
class RolloutStorage(object):
    def __init__(self, num_steps, num_processes, obs_shape):
        self.observations = torch.zeros(num_steps + 1, num_processes, 4)
        self.masks = torch.ones(num_steps + 1, num_processes, 1)
        self.rewards = torch.zeros(num_steps, num_processes, 1)
        self.actions = torch.zeros(num_steps, num_processes, 1).long() # long() 형으로 변환
        # 할인 총 보상 저장
        self.returns = torch.zeros(num_steps + 1, num_processes, 1)
        self.index = 0 # insert할 인덱스
        
    def insert(self, current_obs, action, reward, mask):
        self.observations[self.index + 1].copy_(current_obs)
        self.masks[self.index + 1].copy_(mask)
        self.rewards[self.index].copy_(reward)
        self.actions[self.index].copy_(action)
        
        self.index = (self.index + 1) % NUM_ADVANCED_STEP
        
    def after_update(self):
        # Advantage만큼 학습 단계가 진행되면 가장 새로운 transition을 index0에 저장
        self.observations[0].copy_(self.observations[-1])
        self.masks[0].copy_(self.masks[-1])
        
    def compute_returns(self, next_value):
        # 주의 : 5번쩨 단계 부터 거슬러 올라오며 계산
        # 주의 : 5번째 단계가 Advantage1, 4번째 단계는 Advantage2가 됨
        self.returns[-1] = next_value
        for ad_step in reversed(range(self.rewards.size(0))):
            self.returns[ad_step] = self.returns[ad_step + 1] * GAMMA * self.masks[ad_step + 1] + self.rewards[ad_step]

In [4]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self, n_in, n_mid, n_out):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_in, n_mid)
        self.fc2 = nn.Linear(n_mid, n_mid)
        self.actor = nn.Linear(n_mid, n_out) # 행동을 결정하는 부분이므로 출력개수는 행동의 가짓수
        self.critic = nn.Linear(n_mid, 1) # 상태 가치를 출력하는 부분이므로 출력개수는 1개
        
    def forward(self, x):
        h1 = F.relu(self.fc1(x))
        h2 = F.relu(self.fc2(h1))
        critic_output = self.critic(h2) # 상태가치 계산
        actor_output = self.actor(h2) # 행동가치 계산
        return critic_output, actor_output # 상태가치 출력과 행동가치 출력을 반환함
    
    def act(self, x):
        value, actor_output = self(x)
        action_probs = F.softmax(actor_output, dim=1) # dim=1이므로 행동의 종류에 대해 softmax를 적용
        # multinomial()함수는 행을 기준으로 sampling을 한다. 따라서 (m * n)과 같은 행렬이 있을 때 이 함수를 사용하면 (m * num_samples)처럼 만들어진다.
        # 그리고 위 함수는 샘플링된 값의 인덱스를 반환한다.
        action = action_probs.multinomial(num_samples=1)
        return action
    
    def get_value(self, x):
        value, actor_output = self(x)
        return value
    
    def evaluate_actions(self, x, actions):
        # 상태 x로 부터 상태가치, 실제행동 actions의 로그 확률, 엔트로피를 계산
        value, actor_output = self(x)
        
        # log_softmax는 softmax 함수를 적용하고 그 값에 log를 취한 것이다.
        log_probs = F.log_softmax(actor_output, dim=1) # 행동의 종류에 대해서 확률을 계산
        action_log_probs = log_probs.gather(1, actions)
        
        probs = F.softmax(actor_output, dim=1) # 행동의 종류에 대해서 확률을 계산
        entropy = -(log_probs * probs).sum(-1).mean()
        
        return value, action_log_probs, entropy